In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

# =====================================================
# 1️⃣ 데이터 로드 + 시간 파생
# =====================================================
df = pd.read_csv("new_flight_weather_merged.csv")

df["departure_datetime"] = pd.to_datetime(df["departure_datetime"])
df["dep_hour"] = df["departure_datetime"].dt.hour
df["dep_weekday"] = df["departure_datetime"].dt.weekday
df["is_weekend"] = df["dep_weekday"].isin([5, 6]).astype(int)

print("전체 데이터 수:", len(df))

# =====================================================
# 2️⃣ 컬럼 정의
# =====================================================
num_cols = ["기온(°C)", "풍속_ms", "dep_hour", "dep_weekday", "is_weekend"]
num_cols = [c for c in num_cols if c in df.columns]

cat_cols = ["항공사", "출발지", "도착지", "flight_type"]
cat_cols = [c for c in cat_cols if c in df.columns]

# LightGBM용 category 변환
for c in cat_cols:
    df[c] = df[c].astype("category")

X_cols = num_cols + cat_cols

# =====================================================
# 3️⃣ Train / Test 분리 (시간 기준)
# =====================================================
df = df.sort_values("departure_datetime")
split_date = df["departure_datetime"].quantile(0.8)

train_df = df[df["departure_datetime"] <= split_date]
test_df  = df[df["departure_datetime"] > split_date]

print("Train:", len(train_df), "Test:", len(test_df))

# =====================================================
# 4️⃣ 🔥 다운사이징 (Train 데이터만)
# =====================================================
train_0 = train_df[train_df["is_delay"] == 0]
train_1 = train_df[train_df["is_delay"] == 1]

print("Before Downsampling")
print("정상(0):", len(train_0), "지연(1):", len(train_1))

# 👉 정상(0)을 지연(1) 개수만큼 랜덤 샘플링
train_0_down = train_0.sample(
    n=len(train_1),
    random_state=42
)

train_down = (
    pd.concat([train_0_down, train_1])
    .sample(frac=1, random_state=42)
)

print("After Downsampling")
print(train_down["is_delay"].value_counts())

# =====================================================
# 5️⃣ X / y 분리
# =====================================================
X_train = train_down[X_cols]
y_train = train_down["is_delay"]

X_test  = test_df[X_cols]
y_test  = test_df["is_delay"]

# =====================================================
# 6️⃣ LightGBM 모델 (❌ class_weight 제거)
# =====================================================
lgbm = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,

    objective="binary",
    metric="aucpr",
    random_state=42,
    n_jobs=-1
)

# categorical_feature 지정
lgbm.fit(
    X_train,
    y_train,
    categorical_feature=cat_cols
)

print("✅ LightGBM (Downsampling) 학습 완료")

# =====================================================
# 7️⃣ 평가
# =====================================================
y_prob = lgbm.predict_proba(X_test)[:, 1]

for t in [0.3, 0.35, 0.4, 0.45, 0.5]:
    print(f"\n===== Threshold = {t} =====")
    y_pred = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred))

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))


C:\Users\Admin\AppData\Local\Temp\ipykernel_16464\4080722524.py:9: DtypeWarning: Columns (28,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_flight_weather_merged.csv")


전체 데이터 수: 2843934
Train: 2275147 Test: 568787
Before Downsampling
정상(0): 1938804 지연(1): 336343
After Downsampling
0    336343
1    336343
Name: is_delay, dtype: int64
[LightGBM] [Info] Number of positive: 336343, number of negative: 336343
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013337 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 664
[LightGBM] [Info] Number of data points in the train set: 672686, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
✅ LightGBM (Downsampling) 학습 완료

===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       0.90      0.33      0.48    422243
           1       0.32      0.90      0.47    146544

    accuracy                           0.47    568787
   macro avg       0.61      0.61      0.47    568